In [1]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="http://localhost:12434/engines/v1",
    api_key="docker",
    temperature=0,
    model="ai/gemma3"
)


from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

title='Inception' year=2010 director='Christopher Nolan' rating=8.8


In [2]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]

model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}

{'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}


In [3]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
response
# {
#     "raw": AIMessage(...),
#     "parsed": Movie(title=..., year=..., ...),
#     "parsing_error": None,
# }

{'raw': AIMessage(content='{"title": "Inception", "year": 2010, "director": "Christopher Nolan", "rating": 8.8}\n\n \t \t \t \t \t \t \t \t \t \t', additional_kwargs={'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 16, 'total_tokens': 69, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-SUtkzL7JovVMQvOwxKhdDAS9ELjW49MM', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--57802544-c418-4b06-8ebe-ef3b7e409ce1-0', usage_metadata={'input_tokens': 16, 'output_tokens': 53, 'total_tokens': 69, 'input_token_details': {}, 'output_token_details': {}}),
 'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8),
 'parsing_error': None}

In [6]:
# Nested Structure
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide details about the movie Inception")
response

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role="Dominick 'Dom' Cobb"), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Eames'), Actor(name='Dileep Rao', role='Saito'), Actor(name='Ken Watanabe', role='Yusuf'), Actor(name='Marion Cotillard', role='Mal'), Actor(name='Michael Caine', role='Peter Fischer')], genres=['Action', 'Sci-Fi', 'Thriller'], budget=130000000.0)

In [7]:
# Using TypeDict
from typing_extensions import Annotated, TypedDict

class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: Annotated[float | None, ..., "Budget in millions USD"]

model_with_structure = model.with_structured_output(MovieDetails)

In [8]:
response = model.invoke("Create a picture of a cat")
print(response.content_blocks)
# [
#     {"type": "text", "text": "Here's a picture of a cat"},
#     {"type": "image", "base64": "...", "mime_type": "image/jpeg"},
# ]

[{'type': 'text', 'text': 'Okay, I can\'t *literally* create a picture and display it here. I\'m a text-based AI. However, I can give you a detailed description that you can use to:\n\n1.  **Generate an image with an AI image generator:**  I\'ll provide a prompt you can copy and paste into tools like Midjourney, DALL-E 2, Stable Diffusion, or Bing Image Creator.\n2.  **Draw the cat yourself:**  I\'ll give you a detailed description you can use as a reference.\n3.  **Describe the image to someone else:**  You can tell a friend or family member what you want them to draw or imagine.\n\n\nHere\'s a detailed description of a cat, perfect for generating an image or drawing:\n\n**Image Description: A Relaxed Tabby Cat**\n\n*   **Type of Cat:** A domestic shorthair tabby cat.\n*   **Breed Characteristics:**  Classic tabby markings - swirling patterns of brown, gray, and cream.  The stripes are bold and well-defined.\n*   **Color:** Primarily a warm gray with prominent brown tabby markings.  A

In [9]:
# Reasoning

# Complete reasoning output
response = model.invoke("Why do parrots have colorful feathers?")
reasoning_steps = [b for b in response.content_blocks if b["type"] == "reasoning"]
print(" ".join(step["reasoning"] for step in reasoning_steps))

In [10]:
# Stream reasoning output
for chunk in model.stream("Why do parrots have colorful feathers?"):
    reasoning_steps = [r for r in chunk.content_blocks if r["type"] == "reasoning"]
    print(reasoning_steps if reasoning_steps else chunk.text)


The
 vibrant
 colors
 of
 parrot
 feathers
 are
 a
 fascinating
 result
 of
 a
 complex
 interplay
 of
 genetics
,
 pigments
,
 and
 structural
 coloration
.
 Here
'
s
 a
 breakdown
 of
 the
 key
 reasons
 why
 parrots
 are
 so
 colorful
:



**
1
.
 Pig
ments
 –
 The
 Basics
:**



*
 **
Mel
an
ins
:**
 These
 are
 the
 same
 pigments
 that
 give
 humans
 and
 other
 animals
 dark
 colors
 (
black
,
 brown
,
 gray
).
 Par
rots
 have
 melan
ins
,
 but
 they
'
re
 often
 modified
 and
 used
 in
 subtle
 ways
.


*
 **
Car
oten
oids
:**
 These
 pigments
 are
 derived
 from
 their
 diet
 and
 produce
 yellows
,
 oranges
,
 and
 reds
.
 Par
rots
 get
 these
 from
 fruits
,
 vegetables
,
 and
 seeds
 they
 eat
.
 The
 more
 caroten
oids
 they
 consume
,
 the
 brighter
 the
 color
.


*
 **
P
orph
yr
ins
:**
 These
 pigments
 create
 reds
,
 browns
,
 and
 pur
ples
.
 They
'
re
 produced
 within
 the
 bird
'
s
 body
 and
 are
 often
 linked
 to
 sexual
 maturity
 and
 breeding
.



**
2
.
 

In [20]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="ai/phi4",
    model_provider="openai",
    base_url="http://localhost:12434/engines/v1",
    api_key="docker",
).bind(logprobs=True)

response = model.invoke("Why do parrots talk?")
print(response.response_metadata["logprobs"])

{'content': [{'token': 'Par', 'bytes': [80, 97, 114], 'logprob': -0.01169031672179699, 'top_logprobs': [{'token': 'Par', 'bytes': [80, 97, 114], 'logprob': -0.01169031672179699, 'id': 4368}, {'token': 'As', 'bytes': [65, 115], 'logprob': -4.454984664916992, 'id': 2170}, {'token': 'While', 'bytes': [87, 104, 105, 108, 101], 'logprob': -14.505453109741211, 'id': 8142}, {'token': 'There', 'bytes': [84, 104, 101, 114, 101], 'logprob': -14.586288452148438, 'id': 3947}, {'token': 'The', 'bytes': [84, 104, 101], 'logprob': -14.834005355834961, 'id': 791}, {'token': ' Par', 'bytes': [32, 80, 97, 114], 'logprob': -15.31126594543457, 'id': 4366}, {'token': 'P', 'bytes': [80], 'logprob': -16.218318939208984, 'id': 47}, {'token': '**', 'bytes': [42, 42], 'logprob': -16.575899124145508, 'id': 334}, {'token': 'as', 'bytes': [97, 115], 'logprob': -16.722562789916992, 'id': 300}, {'token': 'par', 'bytes': [112, 97, 114], 'logprob': -17.009450912475586, 'id': 1768}, {'token': 'When', 'bytes': [87, 104,

In [21]:
# Token usage
# Callback handler
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

llm_1 = init_chat_model(
    model="ai/phi4",
    model_provider="openai",
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)
llm_2 = init_chat_model(
    model="ai/gemma3",
    model_provider="openai",
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)

callback = UsageMetadataCallbackHandler()
result_1 = llm_1.invoke("Hello", config={"callbacks": [callback]})
result_2 = llm_2.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata

{'ai/phi4': {'input_tokens': 8,
  'output_tokens': 10,
  'total_tokens': 18,
  'input_token_details': {},
  'output_token_details': {}},
 'ai/gemma3': {'input_tokens': 10,
  'output_tokens': 48,
  'total_tokens': 58,
  'input_token_details': {},
  'output_token_details': {}}}

In [22]:
# Context manager

from langchain.chat_models import init_chat_model
from langchain_core.callbacks import get_usage_metadata_callback

llm_1 = init_chat_model(
    model="ai/phi4",
    model_provider="openai",
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)
llm_2 = init_chat_model(
    model="ai/gemma3",
    model_provider="openai",
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)

with get_usage_metadata_callback() as cb:
    llm_1.invoke("Hello")
    llm_2.invoke("Hello")
    print(cb.usage_metadata)

{'ai/phi4': {'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {}, 'output_token_details': {}}, 'ai/gemma3': {'input_tokens': 10, 'output_tokens': 57, 'total_tokens': 67, 'input_token_details': {}, 'output_token_details': {}}}
